In [0]:
from collections import namedtuple

def inv(n, q):
  return egcd(n, q)[0] % q

def egcd(a, b):
  s0 = 1
  s1 = 0
  t0 = 0
  t1 = 1
  while b > 0:
    q, r = divmod(a, b)
    a, b = b, r
    s0, s1, t0, t1 = s1, s0 - q * s1, t1, t0 - q * t1
  return s0, t0, a

def sqrt(n, q):
  assert n < q
  for i in range(1, q):
    if i * i % q == n:
      return (i, q - i)
  raise ValueError('sqrt not found')

Point = namedtuple('Point', ['x', 'y'])

class EC(object):
  def __init__(self, a, b, q):
    assert 0 < a and a < q and 0 < b and b < q and q > 2
    assert (4 * a**3 + 27 * b**2)  % q != 0
    self.a = a
    self.b = b
    self.q = q
    self.zero = Point(0, 0)

  def is_valid(self, p):
    if p == self.zero:
      return True
    l = (p.y**2) % self.q
    r = ((p.x**3) + self.a * p.x + self.b) % self.q
    return l == r

  def at(self, x):
    assert x < self.q
    ysq = (x**3 + self.a * x + self.b) % self.q
    y, my = sqrt(ysq, self.q)
    return Point(x, y), Point(x, my)

  def neg(self, p):
    return Point(p.x, -p.y % self.q)

  def add(self, p1, p2):
    if p1 == self.zero:
      return p2
    if p2 == self.zero:
      return p1
    if p1.x == p2.x and (p1.y != p2.y or p1.y == 0):
      return self.zero
    if p1.x == p2.x:
      l = (3 * p1.x * p1.x + self.a) * inv(2 * p1.y, self.q) % self.q
    else:
      l = (p2.y - p1.y) * inv(p2.x - p1.x, self.q) % self.q
    x = (l * l - p1.x - p2.x) % self.q
    y = (l * (p1.x - x) - p1.y) % self.q
    return Point(x, y)

  def mul(self, p, n):
    r = self.zero
    m2 = p
    while 0 < n:
      if n & 1 == 1:
        r = self.add(r, m2)
      n, m2 = n >> 1, self.add(m2, m2)
    return r

  def order(self, g):
    assert self.is_valid(g) and g != self.zero
    for i in range(1, self.q + 1):
      if self.mul(g, i) == self.zero:
        return i
    raise ValueError('Invalid order')

class DiffieHellman(object):
  def __init__(self, ec, g):
    self.ec = ec
    self.g = g
    self.n = ec.order(g)

  def gen(self, priv):
    assert 0 < priv and priv < self.n
    return self.ec.mul(self.g, priv)

  def secret(self, priv, pub):
    assert self.ec.is_valid(pub)
    assert self.ec.mul(pub, self.n) == self.ec.zero
    return self.ec.mul(pub, priv)

In [4]:
ec = EC(1, 18, 19)
g, _ = ec.at(7)
assert ec.order(g) <= ec.q

dh = DiffieHellman(ec, g)

apriv = 11
apub = dh.gen(apriv)

bpriv = 3
bpub = dh.gen(bpriv)

cpriv = 7
cpub = dh.gen(cpriv)

apub, bpub, cpub

(Point(x=1, y=1), Point(x=2, y=16), Point(x=15, y=11))

In [5]:
(dh.secret(apriv, bpub) == dh.secret(bpriv, apub)), (dh.secret(apriv, cpub) == dh.secret(cpriv, apub)), (dh.secret(bpriv, cpub) == dh.secret(cpriv, bpub))

(True, True, True)

In [6]:
(dh.secret(apriv, cpub) != dh.secret(apriv, bpub)), (dh.secret(bpriv, apub) != dh.secret(bpriv, cpub)), (dh.secret(cpriv, bpub) != dh.secret(cpriv, apub))

(True, True, True)